# **Machine Learning for Precipitation Nowcasting from Radar Images**

Agrawal, S., Barrington, L., Bromberg, C., Burge, J., Gazen, C., & Hickey, J. (2019). Machine learning for precipitation nowcasting from radar images. arXiv preprint arXiv:1912.12132.

*https://ai.googleblog.com/2020/01/using-machine-learning-to-nowcast.html*

In [1]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [13]:
class HParams(object):
    def __init__(self):
        super(HParams, self).__init__()
        self.w_inp = 7
        self.w_tar = 1

HPARAMS = HParams()

## **Modifed Model**

It is impossible to implement residual connection with the contents contained in thesis and blog. Therefore, short residual connection was not implemented in Textbook-Model.

In [20]:
def BN_LeakyReLU_Conv2D(x, filters, kernel_size = 3, padding = "same", activation_fn = tf.nn.leaky_relu):
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation_fn)(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding = padding)(x)

    return x


def BN_LeakyReLU_MaxPool2D(x, pool_size = 2, pool_padding = "same", activation_fn = tf.nn.leaky_relu):
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation_fn)(x)
    x = tf.keras.layers.MaxPool2D(pool_size, padding = pool_padding)(x)

    return x


def BasicBlock(x, filters, activation_fn = tf.nn.leaky_relu):
    """Conv2D -> BN -> LeakyReLU -> Conv2D"""
    x = tf.keras.layers.Conv2D(filters, 3, padding = "same")(x)
    x = BN_LeakyReLU_Conv2D(x, filters)
    
    return x


def DownsampleBlock(x, filters, activation_fn = tf.nn.leaky_relu):
    """BN -> LeakyReLU -> MaxPool2D -> BN -> LeakyReLU -> Conv2D"""
    x = BN_LeakyReLU_MaxPool2D(x)
    x = BN_LeakyReLU_Conv2D(x, filters)

    return x


def UpsampleBlock(x, filters, activation_fn = tf.nn.leaky_relu):
    """Upsample -> BN -> LeakyReLU -> Conv2D -> BN -> LeakyReLU -> Conv2D"""
    x = tf.keras.layers.UpSampling2D(2, interpolation = "bilinear")(x)
    x = BN_LeakyReLU_Conv2D(x, filters)
    x = BN_LeakyReLU_Conv2D(x, filters)

    return x

In [21]:
IMAGE_SIZE = [256, 256]

def create_NN(model_name = "model", output_dims = 4):
    x = model_input = tf.keras.layers.Input(shape = (*IMAGE_SIZE, HPARAMS.w_inp))

    ## Stem
    x = BasicBlock(x, 32)

    ## Downsampling
    skips = []
    for filters in [64, 128, 128, 256, 256, 512]:
        x = DownsampleBlock(x, filters)
        skips.append(x)

    ## Squeeze
    x = BasicBlock(x, x.shape[-1])

    ## Upsampling
    for filters in [256, 256, 128, 128, 64, 32]:
        x = tf.keras.layers.Concatenate()([x, skips.pop()])
        x = UpsampleBlock(x, filters)

    ## Exit flow
    x = tf.keras.layers.Conv2D(output_dims, 1)(x)
    model_output = x = tf.keras.layers.Activation(tf.nn.softmax)(x)

    return tf.keras.Model(
        inputs = model_input,
        outputs = model_output,
        name = model_name)

In [22]:
tmp = create_NN()
tmp.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 256, 256, 7) 0                                            
__________________________________________________________________________________________________
conv2d_89 (Conv2D)              (None, 256, 256, 32) 2048        input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_107 (BatchN (None, 256, 256, 32) 128         conv2d_89[0][0]                  
__________________________________________________________________________________________________
activation_110 (Activation)     (None, 256, 256, 32) 0           batch_normalization_107[0][0]    
______________________________________________________________________________________________

In [ ]:
# tf.keras.utils.plot_model(tmp, show_shapes = True)

In [17]:
del tmp